# Regression Model For Predicting Movie Box Office Gross

Group Members: Yingjuan Wu, Sahand Zeinali, Ying Wu

Project Description: 


In [1]:
import numpy as np 
import pandas as pd
#import file, remove duplicate movies
movies = pd.read_csv("movie_metadata.csv")
movies = movies.drop_duplicates(['movie_title'])
movies.head()

,color,director_name,num_critic_for_reviews,duration,director_facebook_likes,actor_3_facebook_likes,actor_2_name,actor_1_facebook_likes,gross,genres,...,num_user_for_reviews,language,country,content_rating,budget,title_year,actor_2_facebook_likes,imdb_score,aspect_ratio,movie_facebook_likes
0,Color,James Cameron,723.0,178.0,0.0,855.0,Joel David Moore,1000.0,760505847.0,Action|Adventure|Fantasy|Sci-Fi,...,3054.0,English,USA,PG-13,237000000.0,2009.0,936.0,7.9,1.78,33000
1,Color,Gore Verbinski,302.0,169.0,563.0,1000.0,Orlando Bloom,40000.0,309404152.0,Action|Adventure|Fantasy,...,1238.0,English,USA,PG-13,300000000.0,2007.0,5000.0,7.1,2.35,0
2,Color,Sam Mendes,602.0,148.0,0.0,161.0,Rory Kinnear,11000.0,200074175.0,Action|Adventure|Thriller,...,994.0,English,UK,PG-13,245000000.0,2015.0,393.0,6.8,2.35,85000
3,Color,Christopher Nolan,813.0,164.0,22000.0,23000.0,Christian Bale,27000.0,448130642.0,Action|Thriller,...,2701.0,English,USA,PG-13,250000000.0,2012.0,23000.0,8.5,2.35,164000
4,NaN,Doug Walker,NaN,NaN,131.0,NaN,Rob Walker,131.0,NaN,Documentary,...,NaN,NaN,NaN,NaN,NaN,NaN,12.0,7.1,NaN,0


In [2]:
#select columns
movies = movies[['num_critic_for_reviews', 'duration', 'director_facebook_likes', 'gross', 'genres', 'cast_total_facebook_likes', 'facenumber_in_poster', 'plot_keywords', 'country', 'content_rating', 'budget', 'title_year', 'imdb_score']]

In [3]:
movies.head()

,num_critic_for_reviews,duration,director_facebook_likes,gross,genres,cast_total_facebook_likes,facenumber_in_poster,plot_keywords,country,content_rating,budget,title_year,imdb_score
0,723.0,178.0,0.0,760505847.0,Action|Adventure|Fantasy|Sci-Fi,4834,0.0,avatar|future|marine|native|paraplegic,USA,PG-13,237000000.0,2009.0,7.9
1,302.0,169.0,563.0,309404152.0,Action|Adventure|Fantasy,48350,0.0,goddess|marriage ceremony|marriage proposal|pi...,USA,PG-13,300000000.0,2007.0,7.1
2,602.0,148.0,0.0,200074175.0,Action|Adventure|Thriller,11700,1.0,bomb|espionage|sequel|spy|terrorist,UK,PG-13,245000000.0,2015.0,6.8
3,813.0,164.0,22000.0,448130642.0,Action|Thriller,106759,0.0,deception|imprisonment|lawlessness|police offi...,USA,PG-13,250000000.0,2012.0,8.5
4,NaN,NaN,131.0,NaN,Documentary,143,0.0,NaN,NaN,NaN,NaN,NaN,7.1


In [4]:
#select rows
movies = movies[movies['gross'] > 0]
movies = movies[movies['director_facebook_likes'] > 0]
movies = movies[movies['title_year'] >= 1966]

In [5]:
#print data shape
movies.shape


(3321, 13)

In [6]:
#Categorize Genres feature

#First find out all categories
genresList = movies['genres'].str.split('|')
genresArray = np.asarray(genresList)
oneArray = sum(genresArray, [])
genres = np.unique(oneArray)
#genres = ['Action','Adventure','Animation','Biography','Comedy','Crime','Documentary','Drama','Family','Fantasy','History','Horror','Music','Musical','Mystery','News','Romance','Sci-Fi','Short','Sport','Thriller','War','Western']

#Second add new binary features
i=0 
while i<len(genres): 
    movies[genres[i]]=movies['genres'].str.contains(genres[i]).astype(int)
    i=i+1

#Visualization
movies.head()




,num_critic_for_reviews,duration,director_facebook_likes,gross,genres,cast_total_facebook_likes,facenumber_in_poster,plot_keywords,country,content_rating,...,Musical,Mystery,News,Romance,Sci-Fi,Short,Sport,Thriller,War,Western
1,302.0,169.0,563.0,309404152.0,Action|Adventure|Fantasy,48350,0.0,goddess|marriage ceremony|marriage proposal|pi...,USA,PG-13,...,0,0,0,0,0,0,0,0,0,0
3,813.0,164.0,22000.0,448130642.0,Action|Thriller,106759,0.0,deception|imprisonment|lawlessness|police offi...,USA,PG-13,...,0,0,0,0,0,0,0,1,0,0
5,462.0,132.0,475.0,73058679.0,Action|Adventure|Sci-Fi,1873,1.0,alien|american civil war|male nipple|mars|prin...,USA,PG-13,...,0,0,0,0,1,0,0,0,0,0
7,324.0,100.0,15.0,200807262.0,Adventure|Animation|Comedy|Family|Fantasy|Musi...,2036,1.0,17th century|based on fairy tale|disney|flower...,USA,PG,...,1,0,0,1,0,0,0,0,0,0
9,375.0,153.0,282.0,301956980.0,Adventure|Family|Fantasy|Mystery,58753,3.0,blood|book|love|potion|professor,UK,PG,...,0,1,0,0,0,0,0,0,0,0


In [7]:
#Categorize Countries feature

countries = movies['country'].unique()
i=0
while i<len(countries):
    movies[countries[i]]=movies['country'].str.contains(countries[i]).astype(int)
    i=i+1

movies.head()


,num_critic_for_reviews,duration,director_facebook_likes,gross,genres,cast_total_facebook_likes,facenumber_in_poster,plot_keywords,country,content_rating,...,Brazil,Finland,Norway,Sweden,Argentina,Colombia,Poland,Israel,Indonesia,Afghanistan
1,302.0,169.0,563.0,309404152.0,Action|Adventure|Fantasy,48350,0.0,goddess|marriage ceremony|marriage proposal|pi...,USA,PG-13,...,0,0,0,0,0,0,0,0,0,0
3,813.0,164.0,22000.0,448130642.0,Action|Thriller,106759,0.0,deception|imprisonment|lawlessness|police offi...,USA,PG-13,...,0,0,0,0,0,0,0,0,0,0
5,462.0,132.0,475.0,73058679.0,Action|Adventure|Sci-Fi,1873,1.0,alien|american civil war|male nipple|mars|prin...,USA,PG-13,...,0,0,0,0,0,0,0,0,0,0
7,324.0,100.0,15.0,200807262.0,Adventure|Animation|Comedy|Family|Fantasy|Musi...,2036,1.0,17th century|based on fairy tale|disney|flower...,USA,PG,...,0,0,0,0,0,0,0,0,0,0
9,375.0,153.0,282.0,301956980.0,Adventure|Family|Fantasy|Mystery,58753,3.0,blood|book|love|potion|professor,UK,PG,...,0,0,0,0,0,0,0,0,0,0


In [8]:
#Categorize Content-rating feature

#Replace empty cells or 'Unrated' with 'Not Rated' since they mean the same thing
movies['content_rating'] = movies['content_rating'].replace(np.nan,'Not Rated', regex=True)
movies['content_rating'] = movies['content_rating'].replace('Unrated','Not Rated', regex=True)

rating = movies['content_rating'].unique()
i=0
while i<len(rating):
    movies[rating[i]]=movies['content_rating'].str.match(rating[i]).astype(int)
    i=i+1

movies.head()


,num_critic_for_reviews,duration,director_facebook_likes,gross,genres,cast_total_facebook_likes,facenumber_in_poster,plot_keywords,country,content_rating,...,PG-13,PG,G,R,Not Rated,NC-17,Approved,M,GP,X
1,302.0,169.0,563.0,309404152.0,Action|Adventure|Fantasy,48350,0.0,goddess|marriage ceremony|marriage proposal|pi...,USA,PG-13,...,1,1,0,0,0,0,0,0,0,0
3,813.0,164.0,22000.0,448130642.0,Action|Thriller,106759,0.0,deception|imprisonment|lawlessness|police offi...,USA,PG-13,...,1,1,0,0,0,0,0,0,0,0
5,462.0,132.0,475.0,73058679.0,Action|Adventure|Sci-Fi,1873,1.0,alien|american civil war|male nipple|mars|prin...,USA,PG-13,...,1,1,0,0,0,0,0,0,0,0
7,324.0,100.0,15.0,200807262.0,Adventure|Animation|Comedy|Family|Fantasy|Musi...,2036,1.0,17th century|based on fairy tale|disney|flower...,USA,PG,...,0,1,0,0,0,0,0,0,0,0
9,375.0,153.0,282.0,301956980.0,Adventure|Family|Fantasy|Mystery,58753,3.0,blood|book|love|potion|professor,UK,PG,...,0,1,0,0,0,0,0,0,0,0


In [10]:
#Print out a random row and check if its categorization is correct

#Force to display all columns
pd.set_option('display.max_rows', 200)

#Randomly select a row
movies.iloc[868]


num_critic_for_reviews                                                     105
duration                                                                    88
director_facebook_likes                                                      8
gross                                                              2.81332e+07
genres                               Adventure|Animation|Comedy|Family|Fantasy
cast_total_facebook_likes                                                33822
facenumber_in_poster                                                         1
plot_keywords                ant|miniaturization|punishment|shrinking|sudde...
country                                                                    USA
content_rating                                                              PG
budget                                                                   5e+07
title_year                                                                2006
imdb_score                                          